In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Model, AutoModelForCausalLM
from transformers import pipeline

# Output Vector

In [2]:
model_clm = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [3]:
text = "The king of rock and roll is Elvis Pres"
# text = "import numpy as"
# text = "using System."
# text = "Helsinki is the capital city of"
# text = "Berlin is the capital city of"
# text = "The capital of Germany is"

In [ ]:
inputs = tokenizer(text, return_tensors='pt')
input_ids = inputs.input_ids
input_ids

In [ ]:
import torch 

with torch.no_grad():
    outputs = model_clm(**inputs)

logits = outputs.logits
logits.shape

In [ ]:
logits[0, -1, :5]

In [ ]:
import numpy as np

from torch.nn import functional as F

logits_norm = np.array(F.softmax(outputs.logits[0, -1, :], dim=-1))
logits_norm[:5]

In [ ]:
results = np.argsort(logits_norm)[::-1]

for r in results[:10]:
    print(f"{logits_norm[r] * 100:>6.3f}", "%", tokenizer.decode(r))

In [ ]:
import matplotlib.pyplot as plt

threshold = 0.9

cumsum = np.cumsum(logits_norm[results])
l = len(cumsum[cumsum <= threshold])

x = np.append(logits_norm[results][:l], (1 - cumsum[l - 1]))
labels = [tokenizer.decode(r) for r in  results[:l]] + ["others"]

with plt.ion():
    fig, ax = plt.subplots(figsize=(10, 10))
    _ = ax.pie(x, labels=labels)

# Inference

In [ ]:
generator = pipeline('text-generation', model='gpt2')
output = generator("The capital of germany is")
output[0]["generated_text"]

## Temperature

In [ ]:
output = generator("The capital of germany is", temperature=0.0001)
output[0]["generated_text"]

In [ ]:
output = generator("The capital of germany is", temperature=1)
output[0]["generated_text"]

In [ ]:
output = generator("The capital of germany is", temperature=2.0)
output[0]["generated_text"]

# Top P/K

In [ ]:
output = generator("The capital of germany is", top_p=0.5)
output[0]["generated_text"]

In [ ]:
output = generator("The capital of germany is", top_k=10)
output[0]["generated_text"]

# Trainable Parameters

In [ ]:
print(f"{sum(p.numel() for p in model_clm.parameters() if p.requires_grad):,}")
model_clm.parameters